# Apache Kafkaの環境を構築するためのノードを準備する

Apache Kafka を構築するノードを準備します。

> このNotebookは、VCPを用いずに既存のノードをApache Kafkaの構築環境として利用することを想定しています。そのため、既に[00-101-Apache Kafkaのセットアップ.ipynb](./00-001-%E3%83%8E%E3%83%BC%E3%83%89%E3%81%AE%E8%B5%B7%E5%8B%95.ipynb)を実行してVCノードのセットアップを行っている場合は、**このNotebookを実行する必要はありません**。

ここでは、以下のノードをセットアップすることを想定しています。

* Apache Kafka Broker
  - 1ノード
* Kafka Connector
  - 1ノード
  
また上記の各ノードには、以下の条件を満たすユーザが存在することを前提としています。

* このNotebook環境から、SSHの公開鍵認証でログインできること
* `sudo` によって管理者権限でコマンドを実行できること

# パラメータの指定

## Apache Kafka Broker

Kafka Brokerを実行するノードのホスト名を指定してください。

> Kafka環境を構築するノードのホスト名は名前解決できる必要があります。DNSに登録されているホスト名を指定するか、対応する名前を`/etc/hosts`に登録してください。

In [ ]:
broker_nodes = [
    'broker-0',
]

ログインする際のユーザ名を指定してください。

In [ ]:
# (例)
# broker_user = 'centos'   # CentOSの場合
# broker_user = 'ubuntu'   # Ubuntuの場合

broker_user =

SSHの秘密鍵ファイルのパスを指定してください。

In [ ]:
broker_ssh_private_key = '~/.ssh/id_rsa'

Pythonのパスを指定してください。

In [ ]:
# (例)
# broker_python_interpreter = '/usr/bin/python'   # CentOS7の場合
# broker_python_interpreter = '/usr/bin/python3'  # Ubuntuの場合

broker_python_interpreter =

### Kafkaコネクタ

Kafkaコネクタを実行するノードのホスト名を指定してください。

> 開発用の環境などの場合、Brokerと同じホスト名を指定することも可能です。

In [ ]:
connector_nodes = [
    'connector-0',
]

ログインする際のユーザ名を指定してください。

In [ ]:
# (例)
# connector_user = 'centos'   # CentOSの場合
# connector_user = 'ubuntu'   # Ubuntuの場合

connector_user =

SSHの秘密鍵ファイルのパスを指定してください。

In [ ]:
connector_ssh_private_key = '~/.ssh/id_rsa'

Pythonのパスを指定してください。

In [ ]:
# (例)
# connector_python_interpreter = '/usr/bin/python'   # CentOS7の場合
# connector_python_interpreter = '/usr/bin/python3'  # Ubuntuの場合

connector_python_interpreter =

# Ansibleの設定

操作対象のノードをAnsibleのインベントリに登録します。

In [ ]:
from pathlib import Path

inventory = Path('./hosts')
if inventory.exists():
    inventory.rename(inventory.with_suffix('.orig'))

with inventory.open(mode='w') as f:
    for label in ['broker', 'connector']:
        f.write(f'[{label}]\n')
        for node in eval(f'{label}_nodes'):
            f.write(node)
            f.write('\n')
        f.write(f'''
[{label}:vars]
ansible_user={eval(f"{label}_user")}
ansible_ssh_private_key_file={Path(eval(f"{label}_ssh_private_key")).expanduser()}
ansible_python_interpreter={eval(f"{label}_python_interpreter")}

''')
    
!cat {inventory}
if inventory.with_suffix('.orig').exists():
    try:
        !diff -u {inventory.with_suffix('.orig')} {inventory}
    except:
        pass

`~/.ssh/known_hosts`を更新します。

In [ ]:
for label in ['broker', 'connector']:
    for node in eval(f'{label}_nodes'):
        !ssh-keygen -R {node}
        !ssh-keyscan -H {node} >> ~/.ssh/known_hosts

インベントリのパスを `ansible.cfg` に設定します。

In [ ]:
ansible_cfg = Path('./ansible.cfg')
if ansible_cfg.exists():
    ansible_cfg.rename(ansible_cfg.with_suffix('.orig'))

with ansible_cfg.open(mode='w') as f:
    f.write(f'''
[defaults]
inventory = {inventory.absolute()}
''')
    
!cat {ansible_cfg}
if ansible_cfg.with_suffix('.orig').exists():
    try:
        !diff -u {ansible_cfg.with_suffix('.orig')} {ansible_cfg}
    except:
        pass

疎通確認を行います。

In [ ]:
!ansible all -m ping

# Dockerのインストール

このディレクトリにある一連のNotebookでは、Apache Kafkaの環境を構築するのにDockerコンテナを利用します。ここでは Dockerをインストールします。

> 構築対象のノードに Docker環境が既に構築されている場合は、この章は実行せずに次の章に進んでください。

Dockerのインストール手順はディストリビューション毎に異なります。ここでは CentOS と Ubuntu の場合のセットアップ手順を示します。他のディストリビューションを利用している場合は、[Install Docker CE from binaries](https://docs.docker.com/install/linux/docker-ce/binaries/)などを参考にセットアップを行ってください。

## CentOSの場合

[Get Docker CE for CentOS](https://docs.docker.com/install/linux/docker-ce/centos/)の手順に従いセットアップを行います。

yum のレポジトリ設定を変更して Dockerのレポジトリを追加します。まず、レポジトリを操作するのに必要となるパッケージをインストールします。

In [ ]:
!ansible all -b -m yum -a \
    'name=yum-utils,device-mapper-persistent-data,lvm2 \
    update_cache=yes'

レポジトリを追加します。

In [ ]:
!ansible all -b -a 'yum-config-manager --add-repo https://download.docker.com/linux/centos/docker-ce.repo'

Dockerをインストールします。

In [ ]:
!ansible all -b -m yum -a \
    'name=docker-ce,docker-ce-cli,containerd.io \
    update_cache=yes'

Dockerのサービスを起動します。

In [ ]:
!ansible all -b -m systemd -a 'name=docker enabled=yes state=started daemon-reload=yes'

Dockerがインストールされたことを確認するために `docker info` を実行してみます。

In [ ]:
!ansible all -b -a 'docker info'

管理者権限なしで docker コマンドが実行できるようにするために、ユーザを`docker`グループに所属させます。

In [ ]:
for label in ['broker', 'connector']:
    user = eval(f"{label}_user")
    !ansible {label} -b -m user -a 'name={user} groups=docker append=yes'

## Ubuntuの場合

[Get Docker CE for Ubuntu](https://docs.docker.com/install/linux/docker-ce/ubuntu/)の手順に従いセットアップを行います。

aptのレポジトリ設定を変更して Dockerのレポジトリを追加します。まず、レポジトリを操作するのに必要となるパッケージをインストールします。

In [ ]:
!ansible all -b -m apt -a \
    'name=apt-transport-https,ca-certificates,curl,gnupg-agent,software-properties-common \
    update_cache=yes'

GPGキーを追加します。

In [ ]:
!ansible all -b -m shell -a 'warn=False \
    curl -fsSL https://download.docker.com/linux/ubuntu/gpg | apt-key add -'

レポジトリを追加します。

In [ ]:
!ansible all -b -m shell -a 'add-apt-repository \
    "deb [arch=amd64] https://download.docker.com/linux/ubuntu \
    $(lsb_release -cs) \
    stable"'

Dockerをインストールします。

In [ ]:
!ansible all -b -m apt -a \
    'name=docker-ce,docker-ce-cli,containerd.io \
    update_cache=yes'

Dockerがインストールされたことを確認するために `docker info` を実行してみます。

In [ ]:
!ansible all -b -a 'docker info'

管理者権限なしで docker コマンドが実行できるようにするために、ユーザを`docker`グループに所属させます。

In [ ]:
for label in ['broker', 'connector']:
    user = eval(f"{label}_user")
    !ansible {label} -b -m user -a 'name={user} groups=docker append=yes'

# ツール類のインストール

## 追加パッケージ

環境構築の際に必要となるパッケージをインストールします。

In [ ]:
!ansible all -b -m package -a 'name=rsync'

## docker-compose

In [ ]:
!ansible all -b -m shell -b -a 'warn=False \
    curl -L "https://github.com/docker/compose/releases/download/1.24.0/docker-compose-$(uname -s)-$(uname -m)" \
    -o /usr/bin/docker-compose'
!ansible all -b -a 'warn=False chmod +x /usr/bin/docker-compose'